In [1]:
import os
import sys
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
sys.path.append('../')
from utils import normalize_image, plot_image, plot_images
sys.path.append('/home/li.yu/code/JupiterCVML/europa/base/src/europa')
from dl.config.label_map_helper import LabelConversion, LabelMapHelper
from dl.dataset.farm_field import DriveableTerrainDataset, get_driveable_terrain_dataset
from dl.utils.helpers import load_master_csv

Internal AWS credentials have been removed from brt-devkit.
To setup AWS credentials, Please follow instructions at : https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html


In [2]:
data_dir = '/data2/jupiter/datasets/Jupiter_train_v6_2'
csv_path = '/data2/jupiter/datasets/Jupiter_train_v6_2/master_annotations.csv'
label_map_file = '/home/li.yu/code/JupiterCVML/europa/base/src/europa/dl/config/label_maps/seven_class_train.csv'
# dataset_df = load_master_csv(csv_path)
label_map_helper = LabelMapHelper(label_map_file)
split = 'train'
kwargs = {
    'seed': 304,
    'dataset_name': 'Jupiter_train_v6_2',
    'input_size': [512, 1024],
    'input_mode': 'RGBD',
    'label_map_helper': label_map_helper,
    'val_set_ratio': 0.12,
    'val_csv': '/data2/jupiter/general/v6_2_overlap_with_test_geohash_bag_vat_ids.csv',
    'ignore_label': 255,
    'num_workers' : 16,
    'batch_size': 64,
    'epochs': 60,
    'run_productivity_metrics': False,
    'ignore_dust_with_stop_class': False,
    'n_images_to_sample': 300,
    'night_model': {'use': False},
    'color_jitter': {"use": False, "brightness":0.1,"contrast":0,"saturation":0.2,"hue":0.3},
    'use_albumentation_transform': False,
    'adj_img_gamma': {'use': False},
    'adj_color_temp': {'use': False},
    'trivial_augment': {'use': False},
    'test_time_augmentation': False,
    'normalization_params': {"policy": "tonemap", "alpha": 0.25, "beta": 0.9, "gamma": 0.9, "eps": 1e-6},
    'weighted_sampling': {"birds": 1.0,
                        "mis_labeled_driveable": 0.0, "mis_labeled_driveable_pixels": 10000,
                        "tiny_humans": 0.0, "tiny_human_pixels": 30,
                        "tiny_vehicles": 0.0, "tiny_vehicle_pixels": 100,
                        "humans": 1.0, "human_pixels": [100, 5000],
                        "occluded_humans": 3.5, "occluded_human_pixels": [100, 2000],
                        "reverse_humans": 3.5, "reverse_human_pixels": [50, 2000],
                        "triangle_humans": 3.5, "triangle_human_pixels": [50, 2000],
                        "day_front_vehicles": 1.0, "day_front_vehicle_pixels": [3000, 100000],
                        "day_rear_vehicles": 1.0, "day_rear_vehicle_pixels": [3000, 100000],
                        "night_front_vehicles": 2.0, "night_front_vehicle_pixels": [3000, 100000],
                        "night_rear_vehicles": 2.0, "night_rear_vehicle_pixels": [3000, 100000],
                        "airborne_debris": 3.0, "airborne_debris_pixels": [100, 100000]},
    'cutnpaste_augmentations': {"Humans": {"sample_ratio": 0.3, "human_non_standing_aspect_ratio_threshold": 0.5,
                                           "same_operation_time": False, "same_brightness": True, "brightness_range": 0.05,
                                           "use_standing_human": True, "standing_min_pixels": 20, "standing_max_pixels": 20000,
                                           "use_laying_down_human": True, "laying_down_min_pixels": 50, "laying_down_max_pixels": 15000,
                                           "use_multi_human": True, "only_non_occluded": True, "blend_mode": "vanilla",
                                           "rotate_object": True, "rotate_degree": 30, "jitter_object": False, "jitter_range": 0.15,
                                           "depth_aware": False, "cutout_rate": 0.20, "max_cutout": 0.6},
                                "Tractors or Vehicles": {"sample_ratio": 0.0,
                                           "same_operation_time": False, "same_brightness": True, "brightness_range": 0.05,
                                           "min_pixels": 3000, "max_pixels": 100000, "blend_mode": "gaussianblur",
                                           "rotate_object": False, "rotate_degree": 30, "jitter_object": False, "jitter_range": 0.15,
                                           "depth_aware": False, "cutout_rate": 0.0, "max_cutout": 0.6}},
    'dust_augmentation_imgaug': {'use': False},
}

In [3]:
train_ds, val_ds = get_driveable_terrain_dataset(data_dir, split, csv_path, **kwargs)

/home/li.yu/code/JupiterCVML/europa/base/src/europa/dl/dataset/farm_field.py:465: DtypeWarning: Columns (6,7,39,58,59,60,61,63,64,65,66,67,68,69,70,71,72,73,75,76,77,78,80,81,82,83,84,85,86,87,88,89,90,109,110,111,112,114,115,116,117,118,119,120,121,122,123,124,143,144,145,146,148,149,150,151,152,153,154,155,156,157,158,160,161,162,163,165,166,167,168,169,170,171,172,173,174,175,177,178,179,180,182,183,184,185,186,187,188,189,190,191,192,223,225,227,230,247,251,254,255,261,262,263,267,308,319,324,327,337,338,346,354,370,371,373,374,376,377) have mixed types.Specify dtype option on import or set low_memory=False.
  cur_df = load_master_csv(csv_path)
ERROR:root:Could not parse json for column label_counts in /data2/jupiter/datasets/Jupiter_train_v6_2/master_annotations.csv, it may have been saved on the wrong branch. Dropping it now, it should be regenerated correctly by the cutnpaste_preparer.py step.
/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/pandas/core/in

In [4]:
data_sample = train_ds[0]

In [5]:
sample_df = train_ds.dataframe.iloc[0]
sample_df[['id', 'is_multi_human', 'label_counts', 'humans_cutnpaste_samples']]

id                                                   5f6a93e822ff3085e422c43b
is_multi_human                                                           True
label_counts                      {'Untilled field': 188011, 'Trees': 336277}
humans_cutnpaste_samples    [5f6a75b22f51f8c6c157fba9, 5fe46d297ed99967705...
Name: 0, dtype: object

In [6]:
df = train_ds.dataframe
sample_df = df[df.id == '5f6a75b22f51f8c6c157fba9']
sample_df[['id', 'is_multi_human', 'label_counts', 'humans_cutnpaste_samples']]

,id,is_multi_human,label_counts,humans_cutnpaste_samples
34381,5f6a75b22f51f8c6c157fba9,True,"{'Untilled field': 388089, 'Ignore': 136199}","[5fa4fc10d7c91d86d358c6dd, 5fa0275ecaba9d64166..."


In [2]:
data_dir2 = '/data2/jupiter/datasets/Jupiter_train_v6_1_v2'
df2 = pd.read_csv('/data2/jupiter/datasets/Jupiter_train_v6_1_v2/master_annotations.csv')
# df2 = pd.read_csv('/data/jupiter/li.yu/data/Jupiter_train_v6_1/epoch0_5_30_focal05_master_annotations.csv')
# df2si = pd.read_csv('/data2/jupiter/datasets/Jupiter_train_v6_1_v2/cache/cutnpaste_presampling_4053766482.csv')
# df2 = df2.merge(df2si, on='unique_id')
print(df2.shape)

/home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (6,7,39,223,225,227,230,247,251,254,255,263,267,308,324,327,337,338,346,354,370,371,373,376,377) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(320715, 434)


In [4]:
sample_df2 = df2[df2.unique_id == '5f6a93e822ff3085e422c43b']
sample_df2[['id', 'is_multi_human', 'label_counts', 'humans_cutnpaste_samples']]

,id,is_multi_human,label_counts
2,5f6a93e822ff3085e422c43b,NaN,"{""Untilled field"": 188011, ""Trees"": 336277}"


In [35]:
show_image, show_label = True, True
if show_image:
    data_path = os.path.join(data_dir, sample_df.stereo_pipeline_npz_save_path)
    stereo_data = np.load(data_path)
    img = stereo_data['left']
    img_norm = normalize_image(img, sample_df.hdr_mode)
    print(sample_df.image_id, sample_df.operation_time, sample_df.collected_on)
    plot_image(img_norm)

if show_label and 'rectified_label_save_path' in sample_df and isinstance(sample_df.rectified_label_save_path, str):
    label_path = os.path.join(data_dir, sample_df.rectified_label_save_path)
    lbl = np.load(label_path)['left']
    labels, counts = np.unique(lbl, return_counts=True)
    label_map = sample_df.label_map
    label_strs = [label_map[str(i)] for i in labels if i != 0]
    print(sample_df.unique_id, sample_df.label_counts, label_strs, labels, counts)
    plot_image(lbl)



5f6a93e822ff3085e422c43b daytime 2020-09-15T20:23:19.322000
5f6a93e822ff3085e422c43b {'Untilled field': 188011, 'Trees': 336277} ['Untilled field', 'Trees'] [2 6] [188011 336277]
